In [1]:
!pip install -q kaggle


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Move kaggle.json to ~/.kaggle and set permissions
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# Download and unzip the competition data
!kaggle competitions download -c playground-series-s5e6
!unzip -q playground-series-s5e6.zip -d data/

  0%|                                               | 0.00/11.7M [00:00<?, ?B/s]
100%|██████████████████████████████████████| 11.7M/11.7M [00:00<00:00, 2.91GB/s]


In [4]:
import os
os.listdir('data')

['test.csv', 'train.csv', 'sample_submission.csv']